<a href="https://colab.research.google.com/github/dharanipriyaR24/Machine-Learning/blob/main/NLP%20HUGGINGFACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
train = pd.read_csv("/train.csv")
test = pd.read_csv("/test.csv")
df = pd.concat([train,test],axis = 0)

In [4]:
df['Class Index'].unique() 

array([3, 4, 2, 1])

In [6]:
df['Class Index'] = df['Class Index'].astype('category')
df['Class Index'] = df['Class Index'].cat.codes

In [8]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess(text):
    review = re.sub('[^a-zA-Z]',' ',text) 
    review = review.lower()
    review = review.split()
    #   ps = PorterStemmer()
    lemmatizer=WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(all_stopwords)]
    return ' '.join(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [15]:
df.Title = train.Title.apply(lambda x: preprocess(x))
df.Description = train.Description.apply(lambda x: preprocess(x))

In [16]:
df["Content"] = df['Title'] +" "+ df["Description"]
df.head()

,Class Index,Title,Description,Content
0,0,wall st bear claw back black reuters,reuters short seller wall street dwindling ban...,wall st bear claw back black reuters reuters s...
1,0,carlyle look toward commercial aerospace reuters,reuters private investment firm carlyle group ...,carlyle look toward commercial aerospace reute...
2,0,oil economy cloud stock outlook reuters,reuters soaring crude price plus worry economy...,oil economy cloud stock outlook reuters reuter...
3,0,iraq halt oil export main southern pipeline re...,reuters authority halted oil export flow main ...,iraq halt oil export main southern pipeline re...
4,0,oil price soar time record posing new menace u...,afp tearaway world oil price toppling record s...,oil price soar time record posing new menace u...


In [18]:
df.to_csv('cleaned_merged_df.csv')


In [27]:
df.head()

,Class Index,Title,Description,Content
0,0,wall st bear claw back black reuters,reuters short seller wall street dwindling ban...,wall st bear claw back black reuters reuters s...
1,0,carlyle look toward commercial aerospace reuters,reuters private investment firm carlyle group ...,carlyle look toward commercial aerospace reute...
2,0,oil economy cloud stock outlook reuters,reuters soaring crude price plus worry economy...,oil economy cloud stock outlook reuters reuter...
3,0,iraq halt oil export main southern pipeline re...,reuters authority halted oil export flow main ...,iraq halt oil export main southern pipeline re...
4,0,oil price soar time record posing new menace u...,afp tearaway world oil price toppling record s...,oil price soar time record posing new menace u...


In [28]:
len(df)

127600

In [29]:
from sklearn.model_selection import train_test_split

X = list(df["Content"])
y = list(df["Class Index"])

X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.2,stratify=y)

X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5)


In [30]:
print(len(X_train),len(y_train),len(X_val),len(y_val),len(X_test),len(y_test))

102080 102080 12760 12760 12760 12760


In [31]:
max = 0
for i in X:
    if len(i)>max:
        max = len(i)
    
print(max)

761


In [33]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.2 MB/s eta 0:00:00


In [34]:
import torch
from torch import cuda
from transformers import DistilBertTokenizerFast, TFDistilBertModel


In [35]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = TFDistilBertModel.from_pretrained(model_name)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['classifier', 'dropout_19', 'pre_classifier']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [36]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors = "tf")
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors = "tf")
X_test_tokenized = tokenizer(X_test, padding="max_length", truncation=True, max_length=128, return_tensors = "tf")

In [37]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal 
from tensorflow.keras.losses import CategoricalCrossentropy 
from tensorflow.keras.metrics import CategoricalAccuracy 
from tensorflow.keras.utils import to_categorical

In [38]:
max_len = 128
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len, ), dtype=tf.int32, name="attention_mask")
embeddings = model(input_ids, attention_mask= input_mask)[0] #(0 is the last hidden states, 1 means pooler_output) 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation = 'relu')(out)

y = Dense(4, activation = 'sigmoid')(out)

model = tf.keras.Model(inputs = [input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [39]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 128, 768),                                                   
                                 hidden_states=None                                           

In [46]:
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [43]:
pred_raw = model.predict({'input_ids' :X_test_tokenized['input_ids'], 'attention_mask' :X_test_tokenized['attention_mask']})
y_pred = np.argmax(pred_raw, axis=1)

399/399 [==============================] - 2855s 7s/step


In [48]:
from sklearn.metrics import classification_report
cr = classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.28      0.90      0.42      3158
           1       0.00      0.00      0.00      3172
           2       0.00      0.00      0.00      3213
           3       0.14      0.10      0.12      3217

    accuracy                           0.25     12760
   macro avg       0.10      0.25      0.13     12760
weighted avg       0.10      0.25      0.13     12760



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
